In [1]:
import os
os.environ["OPENAI_API_KEY"]=""

In [3]:
os.environ["GOOGLE_CSE_ID"]=""
os.environ["GOOGLE_API_KEY"]=""

In [4]:
from langchain.retrievers.web_research import WebResearchRetriever
from langchain_chroma import Chroma
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [6]:
# Vectorstore
vectorstore = Chroma(
    embedding_function=OpenAIEmbeddings(), persist_directory="./chroma_db_oai"
)

# LLM
llm = ChatOpenAI(temperature=0)

# Search
search = GoogleSearchAPIWrapper()

In [7]:
search.run("What is vitamin?")

'Dec 16, 2020 ... Fat-soluble vitamins. Vitamins A, D, E, and K are fat-soluble. The body storesTrusted Source fat-soluble vitamins in fatty tissue and the liver,\xa0... Stay hydrated while you enhance your water drinking experience with the vitamins and electrolytes in the many flavors of vitaminwater® . An FNB committee established RDAs for vitamin D to indicate daily intakes sufficient to maintain bone health and normal calcium metabolism in healthy people. Jan 19, 2023 ... Function · Vitamin A helps form and maintain healthy teeth, bones, soft tissue, mucous membranes, and skin. · Vitamin B6 is also called\xa0... Dec 15, 2023 ... The DV for vitamin A is 900 mcg RAE for adults and children age 4 years and older [11], where 1 mcg RAE = 1 mcg retinol, 2 mcg beta-carotene\xa0... Jun 29, 2024 ... Too little B12 in your body can affect your bones, hair, skin, nails, energy level, mood, and memory. You can only get it naturally from animal\xa0... Vitamin A acts as a regulator of cell and 

In [9]:
# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore, llm=llm, search=search
)

ValueError: WebResearchRetriever crawls URLs surfaced through the provided search engine. It is possible that some of those URLs will end up pointing to machines residing on an internal network, leadingto an SSRF (Server-Side Request Forgery) attack. To protect yourself against that risk, you can run the requests through a proxy and prevent the crawler from accidentally crawling internal resources.If've taken the necessary precautions, you can set `allow_dangerous_requests` to `True`.

In [10]:
# Run
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)
from langchain.chains import RetrievalQAWithSourcesChain

user_input = "How do LLM Powered Autonomous Agents work?"
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever
)
result = qa_chain({"question": user_input})
result

NameError: name 'web_research_retriever' is not defined

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.docstore.document import Document
from langchain_community.utilities import ApifyWrapper

apify = ApifyWrapper()
# Call the Actor to obtain text from the crawled webpages
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": [{"url": "/docs/integrations/chat/"}]},
    dataset_mapping_function=lambda item: Document(
        page_content=item["text"] or "", metadata={"source": item["url"]}
    ),
)

# Create a vector store based on the crawled data
index = VectorstoreIndexCreator().from_loaders([loader])

# Query the vector store
query = "Are any OpenAI chat models integrated in LangChain?"
result = index.query(query)
print(result)